In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from PIL import Image
import traceback
import threading
import datetime
import logging
import codecs
import math
import sys
import os


##Uncomment the related dat file ('VesselClassification.dat' for Vessel Classification, 'IMOTrainAndTest.dat' for Vessel Verification/Retrieval/Recognition tasks.)
FILE_TO_DOWNLOAD_FROM = "VesselClassification.dat"
##FILE_TO_DOWNLOAD_FROM = "IMOTrainAndTest.dat" 

NUMBER_OF_WORKERS = 10
MAX_NUM_OF_FILES_IN_FOLDER = 5000
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
ORIGINAL_SIZE = 0 # 1 for yes, 0 for no
JUST_IMAGE = 1 # 1 for yes, 0 for no


photoDetails = ["Photographer:","Title:","Captured:","IMO:","Photo Category:","Description:"]
vesselIdentification = ["Name:","IMO:","Flag:","MMSI:","Callsign:"]
technicalData = ["Vessel type:","Gross tonnage:","Summer DWT:","Length:","Beam:","Draught:"]
additionalInformation = ["Home port:","Class society:","Build year:","Builder (*):","Owner:","Manager:"]
aisInformation = ["Last known position:","Status:","Speed, course (heading):","Destination:","Last update:","Source:"]
impText = photoDetails + vesselIdentification + technicalData + additionalInformation  
impText2 = ["Former name(s):"]

sourceLink = "http://www.shipspotting.com/gallery/photo.php?lid="

logging.basicConfig(level=logging.DEBUG, format='(%(threadName)-10s) %(message)s', )
logging.debug("Process started at " + str(datetime.datetime.now()))

def save_image(ID,justImage,outFolder):
    url = sourceLink + ID
    html = urlopen(url,timeout = 300).read()
    soup = BeautifulSoup(html,"lxml")

    images = [img for img in soup.findAll('img')]
    image_links = [each.get('src') for each in images]
    if not justImage:
        tags = [tr for tr in soup.findAll('td')]
        tr_text = [each.getText() for each in tags]
        
    filename = " "
    for each in image_links:
        if "http" in each and "jpg" in each and "photos/middle" in each:
            filename=each.split('/')[-1]
            f = urlopen(each)
            with open(os.path.join(outFolder,filename), "wb") as local_file:
                local_file.write(f.read())
            if ORIGINAL_SIZE == 0:
                img = Image.open(os.path.join(outFolder,filename)).resize((IMAGE_HEIGHT,IMAGE_WIDTH), Image.ANTIALIAS)
                os.remove(os.path.join(outFolder,filename))
                out = open(os.path.join(outFolder,filename),"wb")
                img.save(out,"JPEG")
            break
        
    if filename != " " and not justImage:
        textFile = filename.split('.')[0]
        tFile = codecs.open(os.path.join(outFolder,filename)+'.dat','w','utf-8')    
        for index,each in enumerate(tr_text):
            for impT in impText:
                if impT == each:
                    tFile.write(each + ' ' + tr_text[index+1] + '\n')
                    break
        for index,each in enumerate(tr_text):
            for impT in impText2:
                if impT == each:
                    for ind in range(1,20):
                        if tr_text[index+ind] != "":
                            tFile.write(each + ' ' + tr_text[index+ind] + '\n')
                        else:
                            break
                    break
        tFile.close()
    if filename == " ":
        return 0
    else:
        return 1


def worker(content,workerNo):
    workerIndex = 0
    folderIndex = 0
    folderNo = 1
    currFolder = os.path.join(os.getcwd(),'W'+str(workerNo)+'_'+str(folderNo))
    if not os.path.exists(currFolder):
        os.mkdir(currFolder)
    for ID in content:
        if folderIndex == MAX_NUM_OF_FILES_IN_FOLDER:
            folderIndex = 0
            folderNo = folderNo + 1
            currFolder = os.path.join(os.getcwd(),'W'+str(workerNo)+'_'+str(folderNo))
            if not os.path.exists(currFolder):
                os.mkdir(currFolder)
        try:
            status = save_image(ID,JUST_IMAGE,currFolder)
            workerIndex = workerIndex + 1
            if status == 1:
                folderIndex = folderIndex + 1
                logging.debug(str(ID) + "\t - Downloaded... - " + str(workerIndex) + "\t/" + str(len(content)))
            else:
                logging.debug(str(ID) + "\t - NO SUCH FILE  - " + str(workerIndex) + "\t/" + str(len(content)))
        except:
            traceback.print_exc()
    logging.debug(str(datetime.datetime.now()) + "-------------- DONE ")
    return

priorFiles = []
dirs = os.listdir(os.getcwd())
for eachDir in dirs:
    if 'W' in eachDir:
        oldFiles = os.listdir(os.path.join(os.getcwd(),eachDir))
        for eachFile in oldFiles:
            if ".jpg" in eachFile:
                oldID = eachFile.split(".")[0]
                priorFiles.append(oldID)

downloadFile = codecs.open(FILE_TO_DOWNLOAD_FROM,"r","utf-8")
downloadContent = downloadFile.readlines()
downloadFile.close()
finalContent = []
for index,eachLine in enumerate(downloadContent):
    temp = eachLine.split(',')[0]
    if temp not in priorFiles:
        finalContent.append(temp)

numOfFiles = len(finalContent)

numOfFilesPerEachWorker = [int(math.floor(float(numOfFiles)/NUMBER_OF_WORKERS)) for x in range(0,NUMBER_OF_WORKERS-1)]
numOfFilesPerEachWorker.append(numOfFiles - (NUMBER_OF_WORKERS-1)*int(round(numOfFiles/NUMBER_OF_WORKERS,0)))

logging.debug("There will be %s workers in this download process" % NUMBER_OF_WORKERS)
logging.debug("%s files will be downloaded" % numOfFiles)

threads = []
imageCount = 0
for i in range(0,NUMBER_OF_WORKERS):
    t = threading.Thread(name='Worker'+str(i), target=worker, args=(finalContent[imageCount:imageCount + numOfFilesPerEachWorker[i]],i,))
    imageCount = imageCount + numOfFilesPerEachWorker[i]
    threads.append(t)
    t.start()

flag = True
while flag:
    counter = 0
    for eachT in threads:
        if eachT.isAlive() == False:
            counter = counter + 1
    if counter == NUMBER_OF_WORKERS:
        flag = False

logging.debug(str(datetime.datetime.now()) + " - list all files startes ")
allPaths = []
allIDs = []
dirs = os.listdir(os.getcwd())
for eachDir in dirs:
    if 'W' in eachDir:
        FinalList = os.listdir(os.path.join(os.getcwd(),eachDir))
        for eachFile in FinalList:
            if ".jpg" in eachFile:
                fPath = os.path.join(os.getcwd(),eachDir,eachFile)
                fID = eachFile.split(".")[0]
                allPaths.append(fPath)
                allIDs.append(fID)
logging.debug(str(datetime.datetime.now()) + " - write to disc ")

FINAL = codecs.open("FINAL.dat","w","utf-8")
for eachLine in downloadContent:
    tempID = eachLine.split(",")[0]
    try:
        tempIndex = allIDs.index(tempID)
        FINAL.write(eachLine[:-1]+","+str(allPaths[tempIndex])+"\n")
    except:
        FINAL.write(eachLine[:-1]+","+"-\n")
FINAL.close()

(MainThread) Process started at 2018-12-13 19:02:36.314095
(MainThread) There will be 10 workers in this download process
(MainThread) 239611 files will be downloaded
Traceback (most recent call last):
  File "<ipython-input-1-d2d829810055>", line 103, in worker
    status = save_image(ID,JUST_IMAGE,currFolder)
  File "<ipython-input-1-d2d829810055>", line 41, in save_image
    html = urlopen(url,timeout = 300).read()
  File "d:\python 3.6.7\Lib\urllib\request.py", line 223, in urlopen
    return opener.open(url, data, timeout)
  File "d:\python 3.6.7\Lib\urllib\request.py", line 526, in open
    response = self._open(req, data)
  File "d:\python 3.6.7\Lib\urllib\request.py", line 544, in _open
    '_open', req)
  File "d:\python 3.6.7\Lib\urllib\request.py", line 504, in _call_chain
    result = func(*args)
  File "d:\python 3.6.7\Lib\urllib\request.py", line 1346, in http_open
    return self.do_open(http.client.HTTPConnection, req)
  File "d:\python 3.6.7\Lib\urllib\request.py", lin

(Worker1   ) 9462	 - Downloaded... - 13	/23961
(Worker3   ) 961577	 - Downloaded... - 14	/23961
(Worker6   ) 2017497	 - Downloaded... - 15	/23961
(Worker0   ) 1498883	 - Downloaded... - 14	/23961
(Worker2   ) 849955	 - Downloaded... - 14	/23961
(Worker8   ) 450955	 - Downloaded... - 16	/23961
(Worker7   ) 1222952	 - Downloaded... - 16	/23961
(Worker5   ) 903012	 - Downloaded... - 15	/23961
(Worker1   ) 861608	 - Downloaded... - 14	/23961
(Worker4   ) 2380904	 - Downloaded... - 14	/23961
(Worker9   ) 2008149	 - Downloaded... - 15	/23962
(Worker3   ) 112456	 - Downloaded... - 15	/23961
(Worker0   ) 955133	 - Downloaded... - 15	/23961
(Worker6   ) 8361	 - Downloaded... - 16	/23961
(Worker2   ) 751153	 - Downloaded... - 15	/23961
(Worker8   ) 2003595	 - Downloaded... - 17	/23961
(Worker5   ) 653542	 - Downloaded... - 16	/23961
(Worker9   ) 1013210	 - Downloaded... - 16	/23962
(Worker4   ) 1634148	 - Downloaded... - 15	/23961
(Worker7   ) 1491970	 - Downloaded... - 17	/23961
(Worker0   ) 64

(Worker1   ) 1328881	 - Downloaded... - 28	/23961
(Worker4   ) 1760689	 - Downloaded... - 29	/23961
(Worker5   ) 2186197	 - Downloaded... - 31	/23961
(Worker2   ) 1135887	 - Downloaded... - 30	/23961
(Worker3   ) 936033	 - Downloaded... - 31	/23961
(Worker6   ) 966149	 - Downloaded... - 31	/23961
(Worker0   ) 1598286	 - Downloaded... - 32	/23961
(Worker4   ) 1514641	 - Downloaded... - 30	/23961
(Worker8   ) 1599463	 - Downloaded... - 33	/23961
(Worker7   ) 151823	 - Downloaded... - 35	/23961
(Worker5   ) 1910961	 - Downloaded... - 32	/23961
(Worker9   ) 1626949	 - Downloaded... - 32	/23962
(Worker3   ) 1789469	 - Downloaded... - 32	/23961
(Worker2   ) 1687725	 - Downloaded... - 31	/23961
(Worker1   ) 2207885	 - Downloaded... - 29	/23961
(Worker6   ) 1917171	 - Downloaded... - 32	/23961
(Worker0   ) 657908	 - Downloaded... - 33	/23961
(Worker4   ) 116676	 - Downloaded... - 31	/23961
(Worker8   ) 881595	 - Downloaded... - 34	/23961
(Worker7   ) 803749	 - Downloaded... - 36	/23961
(Worker

(Worker2   ) 2154535	 - Downloaded... - 46	/23961
(Worker7   ) 1539232	 - Downloaded... - 51	/23961
(Worker4   ) 23531	 - Downloaded... - 44	/23961
(Worker8   ) 635682	 - Downloaded... - 52	/23961
(Worker3   ) 2348458	 - Downloaded... - 48	/23961
(Worker5   ) 720374	 - Downloaded... - 49	/23961
(Worker6   ) 1200137	 - Downloaded... - 48	/23961
(Worker0   ) 1886062	 - Downloaded... - 48	/23961
(Worker9   ) 2016845	 - Downloaded... - 48	/23962
(Worker1   ) 2229888	 - Downloaded... - 44	/23961
(Worker4   ) 405846	 - Downloaded... - 45	/23961
(Worker2   ) 1235218	 - Downloaded... - 47	/23961
(Worker5   ) 602413	 - Downloaded... - 50	/23961
(Worker8   ) 1870045	 - Downloaded... - 53	/23961
(Worker7   ) 1376634	 - Downloaded... - 52	/23961
(Worker6   ) 444349	 - Downloaded... - 49	/23961
(Worker0   ) 692441	 - Downloaded... - 49	/23961
(Worker9   ) 1062517	 - Downloaded... - 49	/23962
(Worker3   ) 1993359	 - Downloaded... - 49	/23961
(Worker4   ) 1775606	 - Downloaded... - 46	/23961
(Worker5

(Worker5   ) 1779508	 - Downloaded... - 82	/23961
(Worker1   ) 895995	 - Downloaded... - 75	/23961
(Worker6   ) 1024374	 - Downloaded... - 79	/23961
(Worker9   ) 1064617	 - Downloaded... - 81	/23962
(Worker8   ) 2349965	 - Downloaded... - 86	/23961
(Worker3   ) 1620079	 - Downloaded... - 80	/23961
(Worker7   ) 673835	 - Downloaded... - 83	/23961
(Worker2   ) 2307310	 - Downloaded... - 77	/23961
(Worker5   ) 1423808	 - Downloaded... - 83	/23961
(Worker1   ) 1645779	 - Downloaded... - 76	/23961
(Worker0   ) 1798891	 - Downloaded... - 81	/23961
(Worker3   ) 813768	 - Downloaded... - 81	/23961
(Worker6   ) 1479746	 - Downloaded... - 80	/23961
(Worker9   ) 1574617	 - Downloaded... - 82	/23962
(Worker4   ) 2274320	 - Downloaded... - 76	/23961
(Worker8   ) 2034276	 - Downloaded... - 87	/23961
(Worker7   ) 1922764	 - Downloaded... - 84	/23961
(Worker1   ) 1115506	 - Downloaded... - 77	/23961
(Worker0   ) 2371173	 - Downloaded... - 82	/23961
(Worker5   ) 1956087	 - Downloaded... - 84	/23961
(Wo

...

(Worker2   ) 76742	 - Downloaded... - 405	/23961
(Worker5   ) 701394	 - Downloaded... - 422	/23961
(Worker9   ) 281800	 - Downloaded... - 434	/23962
(Worker6   ) 1845388	 - Downloaded... - 403	/23961
(Worker8   ) 2119354	 - Downloaded... - 443	/23961
(Worker4   ) 2087571	 - Downloaded... - 404	/23961
(Worker7   ) 1564504	 - Downloaded... - 433	/23961
(Worker1   ) 647503	 - Downloaded... - 401	/23961
(Worker3   ) 2174385	 - Downloaded... - 424	/23961
(Worker5   ) 788854	 - Downloaded... - 423	/23961
(Worker2   ) 1058826	 - Downloaded... - 406	/23961
(Worker9   ) 1968894	 - Downloaded... - 435	/23962
(Worker0   ) 2330258	 - Downloaded... - 430	/23961
(Worker8   ) 301984	 - Downloaded... - 444	/23961
(Worker4   ) 874007	 - Downloaded... - 405	/23961
(Worker7   ) 786067	 - Downloaded... - 434	/23961
(Worker1   ) 121303	 - Downloaded... - 402	/23961
(Worker9   ) 12987	 - Downloaded... - 436	/23962
(Worker5   ) 1910961	 - Downloaded... - 424	/23961
(Worker0   ) 153027	 - Downloaded... - 431	

(Worker8   ) 1889879	 - Downloaded... - 460	/23961
(Worker9   ) 880923	 - Downloaded... - 450	/23962
(Worker7   ) 577315	 - Downloaded... - 448	/23961
(Worker5   ) 261736	 - Downloaded... - 439	/23961
(Worker2   ) 2291931	 - Downloaded... - 420	/23961
(Worker3   ) 2075687	 - Downloaded... - 439	/23961
(Worker4   ) 746509	 - Downloaded... - 422	/23961
(Worker6   ) 1236693	 - Downloaded... - 418	/23961
(Worker5   ) 1266318	 - Downloaded... - 440	/23961
(Worker8   ) 250085	 - Downloaded... - 461	/23961
(Worker0   ) 1221298	 - Downloaded... - 448	/23961
(Worker2   ) 2098918	 - Downloaded... - 421	/23961
(Worker9   ) 1729541	 - Downloaded... - 451	/23962
(Worker1   ) 1399605	 - Downloaded... - 417	/23961
(Worker4   ) 1654362	 - Downloaded... - 423	/23961
(Worker6   ) 1104236	 - Downloaded... - 419	/23961
(Worker3   ) 2345288	 - Downloaded... - 440	/23961
(Worker7   ) 2032198	 - Downloaded... - 449	/23961
(Worker5   ) 2026488	 - Downloaded... - 441	/23961
(Worker8   ) 1192232	 - Downloaded..

(Worker6   ) 1760891	 - Downloaded... - 434	/23961
(Worker0   ) 1818732	 - Downloaded... - 464	/23961
(Worker7   ) 1893610	 - Downloaded... - 465	/23961
(Worker2   ) 1333168	 - Downloaded... - 436	/23961
(Worker4   ) 1909028	 - Downloaded... - 438	/23961
(Worker5   ) 1582486	 - Downloaded... - 455	/23961
(Worker1   ) 2181539	 - Downloaded... - 431	/23961
(Worker8   ) 2016586	 - Downloaded... - 478	/23961
(Worker2   ) 374625	 - Downloaded... - 437	/23961
(Worker9   ) 281365	 - Downloaded... - 466	/23962
(Worker6   ) 1381837	 - Downloaded... - 435	/23961
(Worker3   ) 1943703	 - Downloaded... - 456	/23961
(Worker7   ) 1926720	 - Downloaded... - 466	/23961
(Worker5   ) 1990482	 - Downloaded... - 456	/23961
(Worker4   ) 1651923	 - Downloaded... - 439	/23961
(Worker0   ) 2136060	 - Downloaded... - 465	/23961
(Worker8   ) 1423222	 - Downloaded... - 479	/23961
(Worker2   ) 984315	 - Downloaded... - 438	/23961
(Worker1   ) 1809429	 - Downloaded... - 432	/23961
(Worker3   ) 425731	 - Downloaded.

(Worker1   ) 2360973	 - Downloaded... - 445	/23961
(Worker9   ) 1387052	 - Downloaded... - 482	/23962
(Worker2   ) 669677	 - Downloaded... - 452	/23961
(Worker8   ) 1433042	 - Downloaded... - 496	/23961
(Worker3   ) 242197	 - Downloaded... - 473	/23961
(Worker7   ) 975432	 - Downloaded... - 482	/23961
(Worker6   ) 310289	 - Downloaded... - 450	/23961
(Worker0   ) 1731491	 - Downloaded... - 480	/23961
(Worker4   ) 1709847	 - Downloaded... - 454	/23961
(Worker9   ) 1067988	 - Downloaded... - 483	/23962
(Worker5   ) 2033117	 - Downloaded... - 471	/23961
(Worker1   ) 1720941	 - Downloaded... - 446	/23961
(Worker8   ) 363854	 - Downloaded... - 497	/23961
(Worker2   ) 1143431	 - Downloaded... - 453	/23961
(Worker2   ) 925610	 - NO SUCH FILE  - 454	/23961
(Worker3   ) 188419	 - Downloaded... - 474	/23961
(Worker7   ) 1722041	 - Downloaded... - 483	/23961
(Worker8   ) 209016	 - Downloaded... - 498	/23961
(Worker6   ) 1560720	 - Downloaded... - 451	/23961
(Worker9   ) 1001711	 - Downloaded... -

(Worker4   ) 1476042	 - Downloaded... - 468	/23961
(Worker0   ) 578862	 - Downloaded... - 495	/23961
(Worker1   ) 1947077	 - Downloaded... - 459	/23961
(Worker9   ) 1744835	 - Downloaded... - 501	/23962
(Worker2   ) 1575108	 - Downloaded... - 469	/23961
(Worker3   ) 1670535	 - Downloaded... - 491	/23961
(Worker7   ) 1586585	 - Downloaded... - 498	/23961
(Worker0   ) 1039718	 - Downloaded... - 496	/23961
(Worker8   ) 2065702	 - Downloaded... - 513	/23961
(Worker4   ) 20132	 - Downloaded... - 469	/23961
(Worker5   ) 361588	 - Downloaded... - 486	/23961
(Worker6   ) 504716	 - Downloaded... - 467	/23961
(Worker1   ) 1195640	 - Downloaded... - 460	/23961
(Worker9   ) 1830055	 - Downloaded... - 502	/23962
(Worker9   ) 2276356	 - NO SUCH FILE  - 503	/23962
(Worker8   ) 368267	 - Downloaded... - 514	/23961
(Worker0   ) 152763	 - Downloaded... - 497	/23961
(Worker7   ) 91812	 - Downloaded... - 499	/23961
(Worker2   ) 852801	 - Downloaded... - 470	/23961
(Worker4   ) 2226985	 - Downloaded... - 4